In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

# Problem 27
### For ease of explanation, we will first define some functions for the DFT. While scipy has an fft module which computes the FFT for us, for completeness we will also create our own DFT matrix

In [2]:
from  scipy.fft import fft

def DFT_matrix(N:int)->np.ndarray:
    """
    Function to generate matrix which performs
    DFT on a vector of length N

    Parameters:
    -----------
    N:int
        Length of vector to perform DFT on

    Returns:
    --------
    W: np.ndarray
        DFT matrix

    """
    W = np.zeros((N,N),dtype=complex)
    for l in range(N):
        for m in range(N):
            W[l,m] = np.exp(-2*np.pi*1.j*l*m/N)
    return W

### We now define the relevant functions for DCT-1. We will break this into three components 
1. A straightforward implementation to check ourselves. Just exactly use the formula as written
2. A DFT version which uses our custom function to generate the DFT matrix
3. An FFT version using scipy's fft

In [3]:
def DCT_1_straightforward(x:np.array)->np.array:
    """
    Straightfoward compuation of DCT-1
    without using DFT/FFT trick. Using a list 
    comprehension.

    Parameters:
    ----------
    x: np.array
        Array to compute DCT-1 on 

    Returns:
    --------
    y: np.array
        DCT-1 computed Array
    
    """
    N = len(x)
    y = np.array([x[0]/2+x[-1]*np.power(-1,k)/2+sum([x[j]*np.cos(np.pi/(N-1)*j*k) for j in range(1,N-1)])
                   for k in range(N)])
    return y

def DCT_1_FFT_using_scipy(x:np.array)->np.array:
    """
    Computation of DCT-1 using 
    FFT given by scipy

    Parameters:
    ----------
    x: np.array
        Array to compute DCT-1 on 

    Returns:
    --------
    y: np.array
        DCT-1 computed Array
    
    """
    N= len(x)
    #Generate vector of length 2N-2 for FFT
    fft_vect = np.zeros(2*N-2)
    fft_vect[0] = x[0]/2
    fft_vect[N-1] = x[N-1]/2
    fft_vect[1:N-1] = x[1:N-1]/2
    # [::-1] is just a fancy string indexing way of writing reverse
    fft_vect[N:2*N-2] = x[1:N-1][::-1]/2
    y = fft(fft_vect)[:N]
    return y

def DCT_1_DFT_without_scipy(x:np.array)->np.array:
    """
    DFT computation of DCT-1
    using custom DFT-matrix. 

    Parameters:
    ----------
    x: np.array
        Array to compute DCT-1 on 

    Returns:
    --------
    y: np.array
        DCT-1 computed Array
    
    """
    N= len(x)
    #Generate vector of length 2N-2 for DFT
    fft_vect = np.zeros(2*N-2)
    fft_vect[0] = x[0]/2
    fft_vect[N-1] = x[N-1]/2
    fft_vect[1:N-1] = x[1:N-1]/2
    # [::-1] is just a fancy string indexing way of writing reverse
    fft_vect[N:2*N-2] = x[1:N-1][::-1]/2
    DFT_mat = DFT_matrix(2*N-2)
    y = (DFT_mat@fft_vect)[:N]
    return y

### We now must test that our two implementations actually agree. Our proof indicates this should be the case, but we must check that our implementation agrees. We will use random data to test. 

In [4]:
N = np.random.randint(low=2,high=10)
x = np.random.randn(N)
print(f"Straigtforward agrees with DFT? {np.allclose(DCT_1_straightforward(x),DCT_1_DFT_without_scipy(x))}")
print(f"Straigtforward agrees with FFT? {np.allclose(DCT_1_straightforward(x),DCT_1_FFT_using_scipy(x))}")

Straigtforward agrees with DFT? True
Straigtforward agrees with FFT? True


### Now we may move on to the discrete sine transform (DST-1). Again, we provide 
1. A straightforward implementation to check ourselves. Just exactly use the formula as written
2. A DFT version which uses our custom function to generate the DFT matrix
3. An FFT version using scipy's fft

In [5]:
def DST_1_straightforward(x:np.array)->np.array:
    """
    Straightfoward compuation of DST-1
    without using DFT/FFT trick. Using a list 
    comprehension.

    Parameters:
    ----------
    x: np.array
        Array to compute DST-1 on 

    Returns:
    --------
    y: np.array
        DST-1 computed Array
    
    """
    N = len(x)
    y = np.array([sum([x[j]*np.sin(np.pi/(N+1)*(j+1)*(k+1)) for j in range(N)])
                   for k in range(N)])
    return y

def DST_1_FFT_using_scipy(x:np.array)->np.array:
    """
    Computation of DST-1 using 
    FFT given by scipy

    Parameters:
    ----------
    x: np.array
        Array to compute DCT-1 on 

    Returns:
    --------
    y: np.array
        DCT-1 computed Array
    
    """
    N= len(x)
    #Generate vector of length 2N-2 for FFT
    fft_vect = np.zeros(2*N+2,dtype=complex)
    fft_vect[1:N+1] = x
    # [::-1] is just a fancy string indexing way of writing reverse
    fft_vect[N+2:2*N+2] = -x[::-1]
    y = 1.j/2*fft(fft_vect)[1:N+1]
    return y

def DST_1_DFT_without_scipy(x:np.array)->np.array:
    """
    DFT computation of DST-1
    using custom DFT-matrix. 

    Parameters:
    ----------
    x: np.array
        Array to compute DCT-1 on 

    Returns:
    --------
    y: np.array
        DCT-1 computed Array
    
    """
    N= len(x)
    #Generate vector of length 2N+2 for DFT
    fft_vect = np.zeros(2*N+2,dtype=complex)
    fft_vect[1:N+1] = x
    # [::-1] is just a fancy string indexing way of writing reverse
    fft_vect[N+2:2*N+2] = -x[::-1]
    DFT_mat = DFT_matrix(2*N+2)
    y = 1.j/2*(DFT_mat@fft_vect)[1:N+1]
    return y

### We again must check our implementation. We will again use random data. 

In [6]:
N = np.random.randint(low=2,high=10)
x = np.random.randn(N)
print(f"Straigtforward agrees with DFT? {np.allclose(DST_1_straightforward(x),DST_1_DFT_without_scipy(x))}")
print(f"Straigtforward agrees with FFT? {np.allclose(DST_1_straightforward(x),DST_1_FFT_using_scipy(x))}")

Straigtforward agrees with DFT? True
Straigtforward agrees with FFT? True


### We can even time our functions to check their efficiency and motivate the FFT implementation

In [11]:
N = 3000
x = np.random.randn(N)
time_1 = time.time()
DST_1_straightforward(x)
time_2 = time.time()
time_3 = time.time()
DST_1_FFT_using_scipy(x)
time_4 = time.time()
print(f"Standard took {time_2-time_1} seconds.")
print(f"FFT took {time_4-time_3}")


Standard took 9.624359607696533 seconds.
FFT took 0.0
